진호님이 일단 실행되게끔 만들어 본것

In [1]:
# 3_21 모델학습 후 예측한 결과가 심통치않아 새로 전처리 해보기로 하였다.
# 벡터를 이용한 모델을 생성해서 문자열 값을 리턴 받아보자.
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = tf.keras.utils.get_file( 'test.txt',  'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
train_text = open(train_file,'rb').read().decode(encoding='utf-8')
test_text = open(test_file,'rb').read().decode(encoding='utf-8')



In [2]:
import pandas as pd
#x,y값을 추출후 데이터프레임에 넣는다.
# X 값 추출 피쳐feature, y값 추출target 

df_train = pd.DataFrame({
    'feature':[row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0],
    'target': [ int(row.split('\t')[2]) for row in train_text.split('\n')[1:] if row.count('\t')>0]
})


In [3]:
df_train
# print(y_train[:3])

,feature,target
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...
149995,인간이 문제지.. 소는 뭔죄인가..,0
149996,평점이 너무 낮아서...,1
149997,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [5]:
df_train.iloc[:,:]

,feature,target
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...
149995,인간이 문제지.. 소는 뭔죄인가..,0
149996,평점이 너무 낮아서...,1
149997,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [11]:
#라벨별로 분리해본것
bad = df_train[df_train.iloc[:,1]==0]
good = df_train[df_train.iloc[:,1]==1]


df_train = df_train[:10000]

from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

vectorizer=TfidfVectorizer(tokenizer=Okt().morphs)
X = vectorizer.fit_transform( df_train.iloc[:,0])
y = df_train.iloc[:,1]


c:\Users\uk246\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [12]:
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
import numpy as np
X_train_dense = np.expand_dims( X.toarray(), axis=-1 )
X_train_dense.shape

(10000, 21840, 1)

In [19]:
# X_train_dense.squeeze().shape
# X.toarray().shape
X_train_dense.shape[1]

21840

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input

# 모델 구성deep nueral network
model = Sequential([
    Input(shape=(X_train_dense.shape[1],)),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일 및 학습
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_dense.squeeze(), y, epochs = 10, batch_size = 32, validation_split = 0.2)

Epoch 1/10
250/250 [==============================] - 15s 52ms/step - loss: 0.5527 - accuracy: 0.7514 - val_loss: 0.4342 - val_accuracy: 0.8090
Epoch 2/10
250/250 [==============================] - 13s 51ms/step - loss: 0.2483 - accuracy: 0.9196 - val_loss: 0.4088 - val_accuracy: 0.8035
Epoch 3/10
250/250 [==============================] - 12s 50ms/step - loss: 0.1122 - accuracy: 0.9734 - val_loss: 0.4357 - val_accuracy: 0.7975
Epoch 4/10
250/250 [==============================] - 11s 42ms/step - loss: 0.0583 - accuracy: 0.9891 - val_loss: 0.4708 - val_accuracy: 0.7910
Epoch 5/10
250/250 [==============================] - 12s 48ms/step - loss: 0.0353 - accuracy: 0.9940 - val_loss: 0.5004 - val_accuracy: 0.7925
Epoch 6/10
250/250 [==============================] - 11s 44ms/step - loss: 0.0236 - accuracy: 0.9961 - val_loss: 0.5345 - val_accuracy: 0.7910
Epoch 7/10
250/250 [==============================] - 11s 43ms/step - loss: 0.0170 - accuracy: 0.9970 - val_loss: 0.5633 - val_accuracy:

In [22]:
test = ['나는 너가 싫어']

# 학습된 TF-IDF 벡터라이저로 테스트 데이터 변환
X_test_sample = vectorizer.transform(test)
X_test_sample_dense = X_test_sample.toarray()

# 모델을 사용하여 예측 수행
predictions = model.predict(X_test_sample_dense)
predictions

# 예측 결과 출력 (긍정: 1, 부정: 0)
print("예측 결과:", np.round( predictions ).astype(int))

1/1 [==============================] - 0s 46ms/step
예측 결과: [[1]]


In [ ]:
# model.save('기본 DNN 적용모델.h5')